# The following Notebook is for checking old datasets from Kaggle as well as setting up queries/requests from Steam/SteamSpy for the most up-to-date data.
Please note that there are a lot of cells tagged out for there were many complicated interruptions.

In [ ]:
# Installations required
# pip install steamspypi

In [4]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time
import pickle

# third-party imports
import numpy as np
import pandas as pd
import requests
import steamspypi
from pathlib import Path

# Business Problem: Create a Steam Game - Recommender System

We have two data sets from Kaggle but the data seems to be kind of out-dated. As a safety net, as well as comparison; I will load the data and compare the two. For the reach goal; I will try using some API's to pull the most recent data from Steam or Steam-Affiliated support sites. For a content based recommender system, I will need just the steam dataset. For a collaborative recommender system, I will need the steam dataset and a massive review dataset.
The end goal will be to build a steamlit app which utilizes both Content Based Recommendations and Collaborative Based Recommendations.

# Old Data

In [5]:
steam = pd.read_csv('./data/steam.csv')
steam.head() #AppId/Name/Developer/Required Age, Genres, Categories, Positive vs Negative, Price

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [13]:
steam.shape

(27075, 18)

In [6]:
steam_description_data = pd.read_csv('./data/steam_description_data.csv')
steam_description_data.head() #Only really need one of the three descriptions... probably detailed?

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


In [7]:
steam_media_data = pd.read_csv('./data/steam_media_data.csv')
steam_media_data.head() #Screenshots or picture images to pull from, could be useful later on Steamlit

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN
1,20,https://steamcdn-a.akamaihd.net/steam/apps/20/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN
2,30,https://steamcdn-a.akamaihd.net/steam/apps/30/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/30/...,NaN
3,40,https://steamcdn-a.akamaihd.net/steam/apps/40/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN
4,50,https://steamcdn-a.akamaihd.net/steam/apps/50/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN


In [8]:
steam_requirements_data = pd.read_csv('./data/steam_requirements_data.csv')
steam_requirements_data.head() #This is a bit too extra to have a minimum PC req

,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [9]:
steam_support_info = pd.read_csv('./data/steam_support_info.csv')
steam_support_info.head() #Not sure if I need a support url, doesn't hurt I guess? 	http://steamcommunity.com/app/10	
#Could format everything into the following since it takes me to the steam page for the game

,steam_appid,website,support_url,support_email
0,10,NaN,http://steamcommunity.com/app/10,NaN
1,30,http://www.dayofdefeat.com/,NaN,NaN
2,50,NaN,https://help.steampowered.com,NaN
3,70,http://www.half-life.com/,http://steamcommunity.com/app/70,NaN
4,80,NaN,http://steamcommunity.com/app/80,NaN


In [10]:
steamspy_tag_data = pd.read_csv('./data/steamspy_tag_data.csv')
steamspy_tag_data.head() #Game tags are useful but why are they 144 or 564.... might not need

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550
1,20,0,71,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,122,0,0,0
3,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,77,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
steam_reviews = pd.read_csv('./data/steam_reviews.csv')
steam_reviews.head() #Need for collaborative filtering

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [16]:
steam_reviews.shape

(6417106, 5)

# This is to generate data if data is not available (API)

In [64]:
#The following is an overall function that request data from websites
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    Parameters This is get_request for any website with API.
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [65]:
#Functions for pulling data without frontloading everything and getting blocked by the website domain
def get_cooldown():
    cooldown = 70  # 1 minute plus a cushion
    return cooldown


def get_some_sleep():
    cooldown = get_cooldown()
    print("Sleeping for {} seconds on {}".format(cooldown, time.asctime()))
    time.sleep(cooldown)
    return


def download_a_single_page(page_no=0):
    print("Downloading page={} on {}".format(page_no, time.asctime()))
    data_request = dict()
    data_request["request"] = "all"
    data_request["page"] = str(page_no)
    data = steamspypi.download(data_request)
    return data


def get_file_name(page_no):
    # Get current day as yyyymmdd format
    date_format = "%Y%m%d"
    current_date = time.strftime(date_format)
    file_name = "{}_steamspy_page_{}.json".format(current_date, page_no)
    return file_name


def download_all_pages(num_pages):
    # Download
    for page_no in range(num_pages):
        file_name = get_file_name(page_no)

        if not Path(file_name).is_file():
            page_data = download_a_single_page(page_no=page_no)

            with open(file_name, "w", encoding="utf8") as f:
                json.dump(page_data, f)

            if page_no != (num_pages - 1):
                get_some_sleep()

    # Aggregate

    data = dict()

    for page_no in range(num_pages):
        file_name = get_file_name(page_no)

        with open(file_name, "r", encoding="utf8") as f:
            page_data = json.load(f)

            data.update(page_data)

    return data

In [66]:
#Actual formula that pulls the data from above; run once to create the file.
#if __name__ == "__main__":
#    data = download_all_pages(num_pages=55)
#steam = pd.DataFrame.from_dict(data).transpose().sort_values('appid').reset_index(drop=True)
#print(steam)
#steam.to_csv('./data/download/steamspypidata.csv')

In [67]:
steamdl = pd.read_csv('./data/download/steamspypidata.csv', index_col=0)
steamdl #This is data from SteamSpiPi, really just has AppId, Name, Dev/Pub, Price
#It seems like I'm missing appid compared to the kaggle database, iterating through pages past 55 are empty so I guess
#should combine the appid between this and the kaggle one.

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,852,ValveTestApp852,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,0
1,1002,Rag Doll Kung Fu,Mark Healey,Mark Healey,NaN,56,24,0,"20,000 .. 50,000",0,0,0,0,99.0,99.0,0.0,0
2,1200,Red Orchestra: Ostfront 41-45,Tripwire Interactive,Tripwire Interactive,NaN,1957,296,0,"500,000 .. 1,000,000",262,0,62,0,499.0,499.0,0.0,7
3,1300,SiN Episodes: Emergence,Ritual Entertainment,Ritual Entertainment,NaN,778,92,0,"100,000 .. 200,000",79,0,112,0,999.0,999.0,0.0,2
4,1309,SiN Episodes: Emergence,Ritual Entertainment,Ritual Entertainment,NaN,322,45,0,"200,000 .. 500,000",0,0,0,0,999.0,999.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53958,2135580,Again and Again,Perspektik,Perspektik,NaN,12,0,0,"0 .. 20,000",0,0,0,0,449.0,499.0,10.0,2
53959,2135940,Light Eater,Austin Chase Williams,Austin Chase Williams,NaN,1,0,0,"0 .. 20,000",0,0,0,0,699.0,699.0,0.0,0
53960,2137350,Input Chaos,anthony undreiner,anthony undreiner,NaN,1,0,0,"0 .. 20,000",0,0,0,0,999.0,999.0,0.0,0
53961,2137570,Cairn Stone Balancing,Ammonite Design Studios Ltd,Ammonite Design Studios Ltd,NaN,2,2,0,"0 .. 20,000",0,0,0,0,639.0,799.0,20.0,0


In [149]:
appidnew = steamdl.iloc[:, [0,1,]]
appidold = steam.iloc[:, [0,1,]]
app_list = appidnew.append(appidold)
app_list = app_list.sort_values('appid').drop_duplicates('name').drop_duplicates('appid').reset_index(drop=True)
app_list = app_list.dropna()
app_list.to_csv('./data/download/app_list.csv', index=False)
#This grabs the appid from both the old/new steam dataframes and appends them and removes the duplicates
#We need the appid to start pulling the data for the games itself.

C:\Users\charl\AppData\Local\Temp\ipykernel_22212\4279301091.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  app_list = appidnew.append(appidold)


In [150]:
len(app_list)
#lets see how long our list is 56622 games not bad

56622

In [151]:
#The following script is to start getting data 
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [152]:
#More functions for the sake of data query
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [153]:
#This one actually pulls the data namely process_batches
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
download_path = './data/download'
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=len(app_list),
    batchsize=20
)

Starting at index 0:

Exported lines 0-19 to steam_app_data.csv. Batch 0 time: 0:00:27 (avg: 0:00:27, remaining: 21:29:58)
Exported lines 20-39 to steam_app_data.csv. Batch 1 time: 0:00:27 (avg: 0:00:27, remaining: 21:31:16)
Exported lines 40-59 to steam_app_data.csv. Batch 2 time: 0:00:27 (avg: 0:00:27, remaining: 21:32:26)
Exported lines 60-79 to steam_app_data.csv. Batch 3 time: 0:00:28 (avg: 0:00:27, remaining: 21:33:13)
Exported lines 80-99 to steam_app_data.csv. Batch 4 time: 0:00:27 (avg: 0:00:27, remaining: 21:30:58)
Exported lines 100-119 to steam_app_data.csv. Batch 5 time: 0:00:27 (avg: 0:00:27, remaining: 21:29:16)
Exported lines 120-139 to steam_app_data.csv. Batch 6 time: 0:00:27 (avg: 0:00:27, remaining: 21:27:20)
Exported lines 140-159 to steam_app_data.csv. Batch 7 time: 0:00:27 (avg: 0:00:27, remaining: 21:26:33)
Exported lines 160-179 to steam_app_data.csv. Batch 8 time: 0:00:27 (avg: 0:00:27, remaining: 21:26:19)
Exported lines 180-199 to steam_app_data.csv. Batch 9

No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 1400-1419 to steam_app_data.csv. Batch 70 time: 0:00:58 (avg: 0:00:30, remaining: 23:13:17)
Exported lines 1420-1439 to steam_app_data.csv. Batch 71 time: 0:00:27 (avg: 0:00:30, remaining: 23:10:50)
Exported lines 1440-1459 to steam_app_data.csv. Batch 72 time: 0:00:27 (avg: 0:00:30, remaining: 23:08:30)
Exported lines 1460-1479 to steam_app_data.csv. Batch 73 time: 0:00:28 (avg: 0:00:30, remaining: 23:06:30)
Exported lines 1480-1499 to steam_app_data.csv. Batch 74 time: 0:00:27 (avg: 0:00:30, remaining: 23:04:20)
Exported lines 1500-1519 to steam_app_data.csv. Batch 75 time: 0:00:28 (avg: 0:00:30, remaining: 23:02:24)
Exported lines 1520-1539 to steam_app_data.csv. Batch 76 time: 0:00:27 (avg: 0:00:30, remaining: 23:00:17)
Exported lines 1540-1559 to steam_app_data.csv. Batch 77 time: 0:00:27 (avg: 0:00:30, remaining: 22:58:12)
Exported lines 1560-1579 to steam_app_data.csv. Batch 

Exported lines 4160-4179 to steam_app_data.csv. Batch 208 time: 0:00:27 (avg: 0:00:30, remaining: 21:57:48)
Exported lines 4180-4199 to steam_app_data.csv. Batch 209 time: 0:00:27 (avg: 0:00:30, remaining: 21:56:43)
Exported lines 4200-4219 to steam_app_data.csv. Batch 210 time: 0:00:27 (avg: 0:00:30, remaining: 21:55:40)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 4220-4239 to steam_app_data.csv. Batch 211 time: 0:00:59 (avg: 0:00:30, remaining: 22:01:02)
Exported lines 4240-4259 to steam_app_data.csv. Batch 212 time: 0:00:27 (avg: 0:00:30, remaining: 21:59:52)
Exported lines 4260-4279 to steam_app_data.csv. Batch 213 time: 0:00:27 (avg: 0:00:30, remaining: 21:58:43)
Exported lines 4280-4299 to steam_app_data.csv. Batch 214 time: 0:00:27 (avg: 0:00:30, remaining: 21:57:34)
Exported lines 4300-4319 to steam_app_data.csv. Batch 215 time: 0:00:28 (avg: 0:00:30, remaining: 21:56:42)
E

Exported lines 6860-6879 to steam_app_data.csv. Batch 343 time: 0:00:27 (avg: 0:00:30, remaining: 20:57:38)
Exported lines 6880-6899 to steam_app_data.csv. Batch 344 time: 0:00:27 (avg: 0:00:30, remaining: 20:56:43)
Exported lines 6900-6919 to steam_app_data.csv. Batch 345 time: 0:00:27 (avg: 0:00:30, remaining: 20:55:52)
Exported lines 6920-6939 to steam_app_data.csv. Batch 346 time: 0:00:28 (avg: 0:00:30, remaining: 20:55:05)
Exported lines 6940-6959 to steam_app_data.csv. Batch 347 time: 0:00:27 (avg: 0:00:30, remaining: 20:54:11)
Exported lines 6960-6979 to steam_app_data.csv. Batch 348 time: 0:00:27 (avg: 0:00:30, remaining: 20:53:18)
Exported lines 6980-6999 to steam_app_data.csv. Batch 349 time: 0:00:27 (avg: 0:00:30, remaining: 20:52:25)
Exported lines 7000-7019 to steam_app_data.csv. Batch 350 time: 0:00:27 (avg: 0:00:30, remaining: 20:51:32)
Exported lines 7020-7039 to steam_app_data.csv. Batch 351 time: 0:00:27 (avg: 0:00:30, remaining: 20:50:42)
No response, waiting 10 seco

Exported lines 9580-9599 to steam_app_data.csv. Batch 479 time: 0:00:27 (avg: 0:00:30, remaining: 19:48:13)
Exported lines 9600-9619 to steam_app_data.csv. Batch 480 time: 0:00:27 (avg: 0:00:30, remaining: 19:47:27)
Exported lines 9620-9639 to steam_app_data.csv. Batch 481 time: 0:00:27 (avg: 0:00:30, remaining: 19:46:43)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 9640-9659 to steam_app_data.csv. Batch 482 time: 0:00:58 (avg: 0:00:30, remaining: 19:48:28)
Exported lines 9660-9679 to steam_app_data.csv. Batch 483 time: 0:00:28 (avg: 0:00:30, remaining: 19:47:45)
Exported lines 9680-9699 to steam_app_data.csv. Batch 484 time: 0:00:27 (avg: 0:00:30, remaining: 19:47:00)
Exported lines 9700-9719 to steam_app_data.csv. Batch 485 time: 0:00:27 (avg: 0:00:30, remaining: 19:46:13)
Exported lines 9720-9739 to steam_app_data.csv. Batch 486 time: 0:00:27 (avg: 0:00:30, remaining: 19:45:29)
E

...

Exported lines 18960-18979 to steam_app_data.csv. Batch 948 time: 0:00:28 (avg: 0:00:30, remaining: 15:52:17)
Exported lines 18980-18999 to steam_app_data.csv. Batch 949 time: 0:00:27 (avg: 0:00:30, remaining: 15:51:41)
Exported lines 19000-19019 to steam_app_data.csv. Batch 950 time: 0:00:29 (avg: 0:00:30, remaining: 15:51:07)
Exported lines 19020-19039 to steam_app_data.csv. Batch 951 time: 0:00:27 (avg: 0:00:30, remaining: 15:50:31)
Exported lines 19040-19059 to steam_app_data.csv. Batch 952 time: 0:00:28 (avg: 0:00:30, remaining: 15:49:55)
Exported lines 19060-19079 to steam_app_data.csv. Batch 953 time: 0:00:28 (avg: 0:00:30, remaining: 15:49:20)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 19080-19099 to steam_app_data.csv. Batch 954 time: 0:00:48 (avg: 0:00:30, remaining: 15:49:25)
Exported lines 19100-19119 to steam_app_data.csv. Batch 955 time: 0:00:27 (avg: 0:00:30, remaining: 15:48:47)
Exported lines 19120-19139 to 

Exported lines 21640-21659 to steam_app_data.csv. Batch 1082 time: 0:00:27 (avg: 0:00:30, remaining: 14:44:12)
Exported lines 21660-21679 to steam_app_data.csv. Batch 1083 time: 0:00:27 (avg: 0:00:30, remaining: 14:43:36)
Exported lines 21680-21699 to steam_app_data.csv. Batch 1084 time: 0:00:27 (avg: 0:00:30, remaining: 14:43:01)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 21700-21719 to steam_app_data.csv. Batch 1085 time: 0:00:59 (avg: 0:00:30, remaining: 14:43:16)
Exported lines 21720-21739 to steam_app_data.csv. Batch 1086 time: 0:00:27 (avg: 0:00:30, remaining: 14:42:41)
Exported lines 21740-21759 to steam_app_data.csv. Batch 1087 time: 0:00:28 (avg: 0:00:30, remaining: 14:42:06)
Exported lines 21760-21779 to steam_app_data.csv. Batch 1088 time: 0:00:28 (avg: 0:00:30, remaining: 14:41:31)
Exported lines 21780-21799 to steam_app_data.csv. Batch 1089 time: 0:00:27 (avg: 0:00:30

Exported lines 24280-24299 to steam_app_data.csv. Batch 1214 time: 0:00:27 (avg: 0:00:30, remaining: 13:37:13)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 24300-24319 to steam_app_data.csv. Batch 1215 time: 0:00:59 (avg: 0:00:30, remaining: 13:37:20)
Exported lines 24320-24339 to steam_app_data.csv. Batch 1216 time: 0:00:27 (avg: 0:00:30, remaining: 13:36:45)
Exported lines 24340-24359 to steam_app_data.csv. Batch 1217 time: 0:00:27 (avg: 0:00:30, remaining: 13:36:11)
Exported lines 24360-24379 to steam_app_data.csv. Batch 1218 time: 0:00:27 (avg: 0:00:30, remaining: 13:35:36)
Exported lines 24380-24399 to steam_app_data.csv. Batch 1219 time: 0:00:27 (avg: 0:00:30, remaining: 13:35:02)
Exported lines 24400-24419 to steam_app_data.csv. Batch 1220 time: 0:00:27 (avg: 0:00:30, remaining: 13:34:27)
Exported lines 24420-24439 to steam_app_data.csv. Batch 1221 time: 0:00:33 (avg: 0:00:30

Exported lines 26940-26959 to steam_app_data.csv. Batch 1347 time: 0:00:28 (avg: 0:00:30, remaining: 12:30:48)
Exported lines 26960-26979 to steam_app_data.csv. Batch 1348 time: 0:00:28 (avg: 0:00:30, remaining: 12:30:15)
Exported lines 26980-26999 to steam_app_data.csv. Batch 1349 time: 0:00:27 (avg: 0:00:30, remaining: 12:29:40)
Exported lines 27000-27019 to steam_app_data.csv. Batch 1350 time: 0:00:28 (avg: 0:00:30, remaining: 12:29:08)
Exported lines 27020-27039 to steam_app_data.csv. Batch 1351 time: 0:00:27 (avg: 0:00:30, remaining: 12:28:34)
Exported lines 27040-27059 to steam_app_data.csv. Batch 1352 time: 0:00:27 (avg: 0:00:30, remaining: 12:28:00)
Exported lines 27060-27079 to steam_app_data.csv. Batch 1353 time: 0:00:27 (avg: 0:00:30, remaining: 12:27:26)
Exported lines 27080-27099 to steam_app_data.csv. Batch 1354 time: 0:00:28 (avg: 0:00:30, remaining: 12:26:53)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 1

NameError: name 'SSLError' is not defined

In [117]:
#Ran into an error approximately 27085 games pulled.
#NameError: name 'SSLError' is not defined
#I can't run the list again cause time constraints so I've modified the applist to go onwards from where it stopped
#Thankfully the script saves into excel every batch so I just need to change the app_list and create another steam_app_data2 csv
#Hopefully this doesn't come up as an issue for anyone else

In [157]:
#The following is to create a new list based off where the error occurred
#app_listcheckpoint = app_list
#app_listcheckpoint.to_csv('./data/download/app_listcheckpoint.csv', index=False)

In [158]:
#Looks good
app_listcheckpoint

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force
...,...,...
56618,2135580,Again and Again
56619,2135940,Light Eater
56620,2137350,Input Chaos
56621,2137570,Cairn Stone Balancing


In [162]:
#It got cut off so I had to make a new list onwards
#app_list = app_listcheckpoint[27085:]
#app_list.to_csv('./data/download/app_list.csv', index=False)
#app_list

,appid,name
27086,907330,Amulet Zero 零物语 - Optimize
27087,907350,Welcome To... Chichester 2 - Part I : The Spy ...
27088,907380,The Ballad Singer
27089,907390,Numberline 3
27090,907410,Hard Times
...,...,...
56618,2135580,Again and Again
56619,2135940,Light Eater
56620,2137350,Input Chaos
56621,2137570,Cairn Stone Balancing


In [164]:
#Have to redefine process_batches for the new one and change some parameters.
def process_batches2(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))
    
# Set file parameters
download_path = './data/download'
steam_app_data = 'steam_app_data2.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches2(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=len(app_list),
    batchsize=20
)

Starting at index 0:

Exported lines 0-19 to steam_app_data2.csv. Batch 0 time: 0:00:27 (avg: 0:00:27, remaining: 11:15:13)
Exported lines 20-39 to steam_app_data2.csv. Batch 1 time: 0:00:27 (avg: 0:00:27, remaining: 11:12:14)
Exported lines 40-59 to steam_app_data2.csv. Batch 2 time: 0:00:28 (avg: 0:00:27, remaining: 11:13:21)
Exported lines 60-79 to steam_app_data2.csv. Batch 3 time: 0:00:27 (avg: 0:00:27, remaining: 11:11:01)
Exported lines 80-99 to steam_app_data2.csv. Batch 4 time: 0:00:27 (avg: 0:00:27, remaining: 11:10:41)
Exported lines 100-119 to steam_app_data2.csv. Batch 5 time: 0:00:27 (avg: 0:00:27, remaining: 11:08:56)
Exported lines 120-139 to steam_app_data2.csv. Batch 6 time: 0:00:28 (avg: 0:00:27, remaining: 11:09:35)
Exported lines 140-159 to steam_app_data2.csv. Batch 7 time: 0:00:27 (avg: 0:00:27, remaining: 11:09:17)
Exported lines 160-179 to steam_app_data2.csv. Batch 8 time: 0:00:27 (avg: 0:00:27, remaining: 11:08:45)
Exported lines 180-199 to steam_app_data2.cs

Exported lines 2740-2759 to steam_app_data2.csv. Batch 137 time: 0:00:27 (avg: 0:00:30, remaining: 11:15:38)
Exported lines 2760-2779 to steam_app_data2.csv. Batch 138 time: 0:00:27 (avg: 0:00:30, remaining: 11:14:38)
Exported lines 2780-2799 to steam_app_data2.csv. Batch 139 time: 0:00:27 (avg: 0:00:30, remaining: 11:13:39)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 2800-2819 to steam_app_data2.csv. Batch 140 time: 0:00:58 (avg: 0:00:30, remaining: 11:17:36)
Exported lines 2820-2839 to steam_app_data2.csv. Batch 141 time: 0:00:27 (avg: 0:00:30, remaining: 11:16:34)
Exported lines 2840-2859 to steam_app_data2.csv. Batch 142 time: 0:00:27 (avg: 0:00:30, remaining: 11:15:34)
Exported lines 2860-2879 to steam_app_data2.csv. Batch 143 time: 0:00:27 (avg: 0:00:30, remaining: 11:14:37)
Exported lines 2880-2899 to steam_app_data2.csv. Batch 144 time: 0:00:27 (avg: 0:00:30, remaining: 11:

Exported lines 4080-4099 to steam_app_data2.csv. Batch 204 time: 0:00:28 (avg: 0:00:30, remaining: 10:44:31)
Exported lines 4100-4119 to steam_app_data2.csv. Batch 205 time: 0:00:28 (avg: 0:00:30, remaining: 10:43:46)
Exported lines 4120-4139 to steam_app_data2.csv. Batch 206 time: 0:00:28 (avg: 0:00:30, remaining: 10:43:00)
Exported lines 4140-4159 to steam_app_data2.csv. Batch 207 time: 0:00:28 (avg: 0:00:30, remaining: 10:42:13)
Exported lines 4160-4179 to steam_app_data2.csv. Batch 208 time: 0:00:27 (avg: 0:00:30, remaining: 10:41:24)
Exported lines 4180-4199 to steam_app_data2.csv. Batch 209 time: 0:00:28 (avg: 0:00:30, remaining: 10:40:39)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 4200-4219 to steam_app_data2.csv. Batch 210 time: 0:00:59 (avg: 0:00:30, remaining: 10:43:04)
Exported lines 4220-4239 to steam_app_data2.csv. Batch 211 time: 0:00:28 (avg: 0:00:30, remaining: 10:

Exported lines 5420-5439 to steam_app_data2.csv. Batch 271 time: 0:00:27 (avg: 0:00:30, remaining: 10:12:20)
Exported lines 5440-5459 to steam_app_data2.csv. Batch 272 time: 0:00:27 (avg: 0:00:30, remaining: 10:11:35)
Exported lines 5460-5479 to steam_app_data2.csv. Batch 273 time: 0:00:27 (avg: 0:00:30, remaining: 10:10:51)
Exported lines 5480-5499 to steam_app_data2.csv. Batch 274 time: 0:00:27 (avg: 0:00:30, remaining: 10:10:06)
Exported lines 5500-5519 to steam_app_data2.csv. Batch 275 time: 0:00:27 (avg: 0:00:30, remaining: 10:09:22)
Exported lines 5520-5539 to steam_app_data2.csv. Batch 276 time: 0:00:27 (avg: 0:00:30, remaining: 10:08:38)
Exported lines 5540-5559 to steam_app_data2.csv. Batch 277 time: 0:00:27 (avg: 0:00:30, remaining: 10:07:54)
Exported lines 5560-5579 to steam_app_data2.csv. Batch 278 time: 0:00:27 (avg: 0:00:30, remaining: 10:07:10)
Exported lines 5580-5599 to steam_app_data2.csv. Batch 279 time: 0:00:28 (avg: 0:00:30, remaining: 10:06:28)
No response, waitin

Exported lines 8120-8139 to steam_app_data2.csv. Batch 406 time: 0:00:27 (avg: 0:00:30, remaining: 9:02:51)
Exported lines 8140-8159 to steam_app_data2.csv. Batch 407 time: 0:00:27 (avg: 0:00:30, remaining: 9:02:13)
Exported lines 8160-8179 to steam_app_data2.csv. Batch 408 time: 0:00:27 (avg: 0:00:30, remaining: 9:01:34)
Exported lines 8180-8199 to steam_app_data2.csv. Batch 409 time: 0:00:27 (avg: 0:00:30, remaining: 9:00:55)
Exported lines 8200-8219 to steam_app_data2.csv. Batch 410 time: 0:00:27 (avg: 0:00:30, remaining: 9:00:16)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 8220-8239 to steam_app_data2.csv. Batch 411 time: 0:00:59 (avg: 0:00:30, remaining: 9:00:59)
Exported lines 8240-8259 to steam_app_data2.csv. Batch 412 time: 0:00:27 (avg: 0:00:30, remaining: 9:00:20)
Exported lines 8260-8279 to steam_app_data2.csv. Batch 413 time: 0:00:27 (avg: 0:00:30, remaining: 8:59:42)
E

...

Exported lines 20120-20139 to steam_app_data2.csv. Batch 1006 time: 0:00:27 (avg: 0:00:30, remaining: 3:57:37)
Exported lines 20140-20159 to steam_app_data2.csv. Batch 1007 time: 0:00:27 (avg: 0:00:30, remaining: 3:57:05)
Exported lines 20160-20179 to steam_app_data2.csv. Batch 1008 time: 0:00:27 (avg: 0:00:30, remaining: 3:56:34)
Exported lines 20180-20199 to steam_app_data2.csv. Batch 1009 time: 0:00:28 (avg: 0:00:30, remaining: 3:56:02)
Exported lines 20200-20219 to steam_app_data2.csv. Batch 1010 time: 0:00:28 (avg: 0:00:30, remaining: 3:55:31)
Exported lines 20220-20239 to steam_app_data2.csv. Batch 1011 time: 0:00:27 (avg: 0:00:30, remaining: 3:54:59)
Exported lines 20240-20259 to steam_app_data2.csv. Batch 1012 time: 0:00:27 (avg: 0:00:30, remaining: 3:54:27)
Exported lines 20260-20279 to steam_app_data2.csv. Batch 1013 time: 0:00:27 (avg: 0:00:30, remaining: 3:53:55)
Exported lines 20280-20299 to steam_app_data2.csv. Batch 1014 time: 0:00:27 (avg: 0:00:30, remaining: 3:53:23)
N

Exported lines 21460-21479 to steam_app_data2.csv. Batch 1073 time: 0:00:27 (avg: 0:00:30, remaining: 3:23:32)
Exported lines 21480-21499 to steam_app_data2.csv. Batch 1074 time: 0:00:27 (avg: 0:00:30, remaining: 3:23:01)
Exported lines 21500-21519 to steam_app_data2.csv. Batch 1075 time: 0:00:27 (avg: 0:00:30, remaining: 3:22:29)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 21520-21539 to steam_app_data2.csv. Batch 1076 time: 0:00:58 (avg: 0:00:30, remaining: 3:22:09)
Exported lines 21540-21559 to steam_app_data2.csv. Batch 1077 time: 0:00:27 (avg: 0:00:30, remaining: 3:21:37)
Exported lines 21560-21579 to steam_app_data2.csv. Batch 1078 time: 0:00:27 (avg: 0:00:30, remaining: 3:21:06)
Exported lines 21580-21599 to steam_app_data2.csv. Batch 1079 time: 0:00:27 (avg: 0:00:30, remaining: 3:20:34)
Exported lines 21600-21619 to steam_app_data2.csv. Batch 1080 time: 0:00:28 (avg: 0:00:3

Exported lines 24100-24119 to steam_app_data2.csv. Batch 1205 time: 0:00:27 (avg: 0:00:30, remaining: 2:16:46)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 24120-24139 to steam_app_data2.csv. Batch 1206 time: 0:00:58 (avg: 0:00:30, remaining: 2:16:22)
Exported lines 24140-24159 to steam_app_data2.csv. Batch 1207 time: 0:00:28 (avg: 0:00:30, remaining: 2:15:51)
Exported lines 24160-24179 to steam_app_data2.csv. Batch 1208 time: 0:00:27 (avg: 0:00:30, remaining: 2:15:20)
Exported lines 24180-24199 to steam_app_data2.csv. Batch 1209 time: 0:00:27 (avg: 0:00:30, remaining: 2:14:49)
Exported lines 24200-24219 to steam_app_data2.csv. Batch 1210 time: 0:00:27 (avg: 0:00:30, remaining: 2:14:18)
Exported lines 24220-24239 to steam_app_data2.csv. Batch 1211 time: 0:00:27 (avg: 0:00:30, remaining: 2:13:47)
Exported lines 24240-24259 to steam_app_data2.csv. Batch 1212 time: 0:00:27 (avg: 0:00:3

Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 26720-26739 to steam_app_data2.csv. Batch 1336 time: 0:00:58 (avg: 0:00:30, remaining: 1:10:44)
Exported lines 26740-26759 to steam_app_data2.csv. Batch 1337 time: 0:00:27 (avg: 0:00:30, remaining: 1:10:14)
Exported lines 26760-26779 to steam_app_data2.csv. Batch 1338 time: 0:00:27 (avg: 0:00:30, remaining: 1:09:43)
Exported lines 26780-26799 to steam_app_data2.csv. Batch 1339 time: 0:00:27 (avg: 0:00:30, remaining: 1:09:13)
Exported lines 26800-26819 to steam_app_data2.csv. Batch 1340 time: 0:00:27 (avg: 0:00:30, remaining: 1:08:42)
Exported lines 26820-26839 to steam_app_data2.csv. Batch 1341 time: 0:00:27 (avg: 0:00:30, remaining: 1:08:11)
Exported lines 26840-26859 to steam_app_data2.csv. Batch 1342 time: 0:00:28 (avg: 0:00:30, remaining: 1:07:41)
Exported lines 26860-26879 to steam_app_data2.csv. Batch 1343 time: 0:00:27 (avg: 0:00:30, remaining: 1:07:10)
Exported lines 26880-26899 to steam_app_data2.csv. Batch 

Exported lines 29360-29379 to steam_app_data2.csv. Batch 1468 time: 0:00:27 (avg: 0:00:30, remaining: 0:04:03)
Exported lines 29380-29399 to steam_app_data2.csv. Batch 1469 time: 0:00:27 (avg: 0:00:30, remaining: 0:03:32)
Exported lines 29400-29419 to steam_app_data2.csv. Batch 1470 time: 0:00:27 (avg: 0:00:30, remaining: 0:03:02)
Exported lines 29420-29439 to steam_app_data2.csv. Batch 1471 time: 0:00:27 (avg: 0:00:30, remaining: 0:02:32)
Exported lines 29440-29459 to steam_app_data2.csv. Batch 1472 time: 0:00:27 (avg: 0:00:30, remaining: 0:02:01)
Exported lines 29460-29479 to steam_app_data2.csv. Batch 1473 time: 0:00:27 (avg: 0:00:30, remaining: 0:01:31)
Exported lines 29480-29499 to steam_app_data2.csv. Batch 1474 time: 0:00:27 (avg: 0:00:30, remaining: 0:01:01)
Exported lines 29500-29519 to steam_app_data2.csv. Batch 1475 time: 0:00:27 (avg: 0:00:30, remaining: 0:00:30)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 1

In [ ]:
#Missed the middle apparently, one more query
#app_list = app_listcheckpoint[20438:27085]
#app_list.to_csv('./data/download/app_list.csv', index=False)

In [129]:
#Checking the data from the first part before the error; few errors have to specify dtype or something
steamappdata = pd.read_csv('./data/download/steam_app_data.csv', error_bad_lines=False, skip_blank_lines=False)
steamappdata

C:\Users\charl\AppData\Local\Temp\ipykernel_22212\3844373562.py:1: DtypeWarning: Columns (2,3,10,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,243,244,245,246,247,248,252,253,255,256,257,258,260,261,262,263,264,266,267,268,269,270) have mixed types. Specify dtype option on import or set low_mem

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,Unnamed: 261,Unnamed: 262,Unnamed: 263,Unnamed: 264,Unnamed: 265,Unnamed: 266,Unnamed: 267,Unnamed: 268,Unnamed: 269,Unnamed: 270
0,game,Counter-Strike,10,0,FALSE,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,Team Fortress Classic,20,0,FALSE,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,game,Day of Defeat,30,0,FALSE,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,game,Deathmatch Classic,40,0,FALSE,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,game,Half-Life: Opposing Force,50,0,FALSE,NaN,NaN,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20794,game,Mega Man 11,742300,0.0,False,full,"[1001110, 861200]",Mega Man is back! The newest entry in this ico...,Mega Man is back! The newest entry in this ico...,Mega Man is back! The newest entry in this ico...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20795,game,Super BoxMan Ultra,742340,0.0,False,full,NaN,<strong>Super BoxMan Ultra</strong> is the qui...,<strong>Super BoxMan Ultra</strong> is the qui...,Super BoxMan Ultra is the quintessential Sokob...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20796,game,全民王者,742360,0.0,False,NaN,NaN,《全民王者》是一款三国题材轻FTG端游。该游戏支持多人连线，以第三视角向玩家展现武将飒爽英姿...,《全民王者》是一款三国题材轻FTG端游。该游戏支持多人连线，以第三视角向玩家展现武将飒爽英姿...,《全民王者》是一款三国题材轻FTG端游。游戏主打PVP游戏模式，玩家通过游戏剧情主线磨砺自己...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20797,game,Spreadstorm,742380,0.0,False,NaN,NaN,Spreadstorm is a single player adventure game ...,Spreadstorm is a single player adventure game ...,Spreadstorm is a puzzle adventure game in whic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
#Data from the second half for merging purposes
steamappdata2 = pd.read_csv('./data/download/steam_app_data2.csv', error_bad_lines=False, skip_blank_lines=False)
steamappdata2

C:\Users\charl\AppData\Local\Temp\ipykernel_22212\1668342952.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  steamappdata2 = pd.read_csv('./data/download/steam_app_data2.csv', error_bad_lines=False, skip_blank_lines=False)


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Amulet Zero 零物语 - Optimize,907330,0,False,NaN,NaN,<h1>联系交流</h1><p>如果您无法正常进入steam社区交流，<br>欢迎加入我们的...,"<img src=""https://cdn.akamai.steamstatic.com/s...",某一天，一觉醒来的朵琳，发现自己正置身于这个陌生的小岛上， 独自在树林里走着，没走多远，她遇...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '23', 'description': 'Indie'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256723294, 'name': '游戏内容展示', 'thumbnai...",NaN,"{'total': 21, 'highlighted': [{'name': '#1 朋友'...","{'coming_soon': False, 'date': 'Aug 14, 2018'}","{'url': '', 'email': 'ciella@qq.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,Welcome To... Chichester 2 - Part I : The Spy ...,907350,0,True,NaN,"[931600, 1021250, 1052780]",<strong><u>About This Game</u></strong><br><br...,<strong><u>About This Game</u></strong><br><br...,Having trouble adjusting to the protagonist's ...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256761818, 'name': 'Trailer', 'thumbna...",NaN,"{'total': 32, 'highlighted': [{'name': 'Chapte...","{'coming_soon': False, 'date': 'Aug 3, 2018'}","{'url': '', 'email': 'triority_developer@outlo...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [1, 2, 5], 'notes': '- Comedic violenc..."
2,game,The Ballad Singer,907380,0,False,full,[928190],"The Ballad Singer is an RPG/Graphic Adventure,...","The Ballad Singer is an RPG/Graphic Adventure,...","The Ballad Singer is an asymmetric adventure, ...",...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256729773, 'name': 'Launch trailer', '...",NaN,"{'total': 40, 'highlighted': [{'name': 'BOOOOO...","{'coming_soon': False, 'date': 'Feb 15, 2019'}","{'url': '', 'email': 'support@curtelgames.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [1, 5], 'notes': 'The game includes fa..."
3,game,Numberline 3,907390,0,False,NaN,NaN,<h1>Adventure and romance! Wishlist and follow...,"<img src=""https://cdn.akamai.steamstatic.com/s...","Numberline 3 is hard, but very interesting puz...",...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256723038, 'name': 'main', 'thumbnail'...",NaN,"{'total': 7, 'highlighted': [{'name': 'Ready t...","{'coming_soon': False, 'date': 'Aug 3, 2018'}","{'url': '', 'email': 'droidriotstudio@gmail.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,game,Hard Times,907410,0,True,NaN,NaN,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",Hard Times is an extreme urban survival game w...,...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256753460, 'name': 'Pre Alpha Trailer'...",NaN,NaN,"{'coming_soon': False, 'date': 'Mar 30, 2020'}","{'url': 'https://forum.tempi-duri.net/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [5], 'notes': 'This game is a street l..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29532,game,Again and Again,2135580,0,False,NaN,NaN,Again and Again is exploratory and slowly buil...,Again and Again is exploratory and slowly buil...,Again and Again is a first-person psychologica...,...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '23', 'description': 'Indie'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256902971,

In [166]:
#Missing line in the middle, download the old one as a checkpoint and create a new app_list
#app_listcheckpoint2 = app_list
#app_listcheckpoint2.to_csv('./data/download/app_listcheckpoint2.csv', index=False)

In [ ]:
#Missing lines in the middle, had to make another query
#app_list = app_listcheckpoint[20438:27085]
#app_list.to_csv('./data/download/app_list.csv', index=False)

In [170]:
#Have to redefine process_batches for the new one and change some parameters.
def process_batches3(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))
    
# Set file parameters
download_path = './data/download'
steam_app_data = 'steam_app_data3.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches3(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=len(app_list),
    batchsize=20
)

Starting at index 0:

Exported lines 0-19 to steam_app_data3.csv. Batch 0 time: 0:00:28 (avg: 0:00:28, remaining: 2:37:03)
Exported lines 20-39 to steam_app_data3.csv. Batch 1 time: 0:00:28 (avg: 0:00:28, remaining: 2:34:28)
Exported lines 40-59 to steam_app_data3.csv. Batch 2 time: 0:00:28 (avg: 0:00:28, remaining: 2:33:41)
Exported lines 60-79 to steam_app_data3.csv. Batch 3 time: 0:00:27 (avg: 0:00:28, remaining: 2:32:29)
Exported lines 80-99 to steam_app_data3.csv. Batch 4 time: 0:00:27 (avg: 0:00:28, remaining: 2:31:36)
Exported lines 100-119 to steam_app_data3.csv. Batch 5 time: 0:00:27 (avg: 0:00:28, remaining: 2:30:21)
Exported lines 120-139 to steam_app_data3.csv. Batch 6 time: 0:00:28 (avg: 0:00:28, remaining: 2:29:54)
Exported lines 140-159 to steam_app_data3.csv. Batch 7 time: 0:00:27 (avg: 0:00:28, remaining: 2:29:16)
Exported lines 160-179 to steam_app_data3.csv. Batch 8 time: 0:00:28 (avg: 0:00:28, remaining: 2:28:51)
Exported lines 180-199 to steam_app_data3.csv. Batch 

No response, waiting 10 seconds...
Retrying.
Exported lines 1400-1419 to steam_app_data3.csv. Batch 70 time: 0:00:58 (avg: 0:00:31, remaining: 2:13:21)
Exported lines 1420-1439 to steam_app_data3.csv. Batch 71 time: 0:00:31 (avg: 0:00:31, remaining: 2:12:51)
Exported lines 1440-1459 to steam_app_data3.csv. Batch 72 time: 0:00:28 (avg: 0:00:31, remaining: 2:12:10)
Exported lines 1460-1479 to steam_app_data3.csv. Batch 73 time: 0:00:28 (avg: 0:00:30, remaining: 2:11:32)
Exported lines 1480-1499 to steam_app_data3.csv. Batch 74 time: 0:00:28 (avg: 0:00:30, remaining: 2:10:54)
Exported lines 1500-1519 to steam_app_data3.csv. Batch 75 time: 0:00:31 (avg: 0:00:30, remaining: 2:10:26)
Exported lines 1520-1539 to steam_app_data3.csv. Batch 76 time: 0:00:27 (avg: 0:00:30, remaining: 2:09:45)
Exported lines 1540-1559 to steam_app_data3.csv. Batch 77 time: 0:00:29 (avg: 0:00:30, remaining: 2:09:10)
Exported lines 1560-1579 to steam_app_data3.csv. Batch 78 time: 0:00:28 (avg: 0:00:30, remaining: 2

No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 2800-2819 to steam_app_data3.csv. Batch 140 time: 0:00:59 (avg: 0:00:31, remaining: 1:37:46)
Exported lines 2820-2839 to steam_app_data3.csv. Batch 141 time: 0:00:28 (avg: 0:00:31, remaining: 1:37:12)
Exported lines 2840-2859 to steam_app_data3.csv. Batch 142 time: 0:00:28 (avg: 0:00:31, remaining: 1:36:38)
Exported lines 2860-2879 to steam_app_data3.csv. Batch 143 time: 0:00:28 (avg: 0:00:30, remaining: 1:36:03)
Exported lines 2880-2899 to steam_app_data3.csv. Batch 144 time: 0:00:28 (avg: 0:00:30, remaining: 1:35:29)
Exported lines 2900-2919 to steam_app_data3.csv. Batch 145 time: 0:00:28 (avg: 0:00:30, remaining: 1:34:56)
Exported lines 2920-2939 to steam_app_data3.csv. Batch 146 time: 0:00:27 (avg: 0:00:30, remaining: 1:34:21)
Exported lines 2940-2959 to steam_app_data3.csv. Batch 147 time: 0:00:27 (avg: 0:00:30, remaining: 1:33:47)
E

Exported lines 4180-4199 to steam_app_data3.csv. Batch 209 time: 0:00:27 (avg: 0:00:30, remaining: 1:02:19)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 4200-4219 to steam_app_data3.csv. Batch 210 time: 0:00:48 (avg: 0:00:30, remaining: 1:01:59)
Exported lines 4220-4239 to steam_app_data3.csv. Batch 211 time: 0:00:28 (avg: 0:00:30, remaining: 1:01:27)
Exported lines 4240-4259 to steam_app_data3.csv. Batch 212 time: 0:00:28 (avg: 0:00:30, remaining: 1:00:55)
Exported lines 4260-4279 to steam_app_data3.csv. Batch 213 time: 0:00:28 (avg: 0:00:30, remaining: 1:00:23)
Exported lines 4280-4299 to steam_app_data3.csv. Batch 214 time: 0:00:28 (avg: 0:00:30, remaining: 0:59:51)
Exported lines 4300-4319 to steam_app_data3.csv. Batch 215 time: 0:00:28 (avg: 0:00:30, remaining: 0:59:20)
Exported lines 4320-4339 to steam_app_data3.csv. Batch 216 time: 0:00:28 (avg: 0:00:30, remaining: 0:58:48)
Exported lines 4340-4359 to steam_app_data3.cs

In [171]:
#Checking the data from the first part before the error; few errors have to specify dtype or something
steamappdata3 = pd.read_csv('./data/download/steam_app_data3.csv', error_bad_lines=False, skip_blank_lines=False)
steamappdata3

C:\Users\charl\AppData\Local\Temp\ipykernel_22212\3962120385.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  steamappdata3 = pd.read_csv('./data/download/steam_app_data3.csv', error_bad_lines=False, skip_blank_lines=False)


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Chronicles of cyberpunk,742470,0.0,True,NaN,NaN,Chronicles of cyberpunk is an adventure in the...,Chronicles of cyberpunk is an adventure in the...,Chronicles of cyberpunk is an adventure in the...,...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256700923, 'name': 'Chronicles of cybe...",NaN,NaN,"{'coming_soon': False, 'date': 'Dec 7, 2017'}","{'url': '', 'email': 'dmitry.kozyr.ua@gmail.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,The Big Journey,742480,0.0,False,full,[789080],<strong>The Big Journey</strong> is a beautifu...,<strong>The Big Journey</strong> is a beautifu...,A cat who's hungry for good food and excitement.,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256706236, 'name': 'The Big Journey Tr...",{'total': 227},"{'total': 31, 'highlighted': [{'name': 'Kitten...","{'coming_soon': False, 'date': 'Apr 20, 2018'}","{'url': 'https://hook.gg/', 'email': 'hello@ho...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,game,Nonogram - The Greatest Painter,742490,0.0,False,NaN,NaN,"<h1>Features</h1><p><img src=""https://cdn.akam...","<i> Nonograms, also known as Picross or Griddl...",Do you deserve to be called The Greatest?,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256711567, 'name': 'Launch trailer', '...",{'total': 221},"{'total': 26, 'highlighted': [{'name': 'A', 'p...","{'coming_soon': False, 'date': 'Mar 22, 2018'}","{'url': 'https://hook.gg/', 'email': 'hello@ho...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,Nonogram - Master's Legacy,742500,0.0,True,NaN,"[957760, 957761, 957762]","<h1>The Greatest pack</h1><p><a href=""https://...","<i> Nonograms, also known as Picross or Griddl...",Master's Legacy - discover all beauty of Nonog...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256734906, 'name': 'Launch trailer', '...",NaN,"{'total': 6, 'highlighted': [{'name': 'First B...","{'coming_soon': False, 'date': 'Nov 8, 2018'}","{'url': 'https://hook.gg/', 'email': 'hello@ho...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,game,Psikodelya,742510,13.0,False,NaN,"[1060200, 1286630]","<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",Explore multiple dimensions in a psychedelic w...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256758843, 'name': 'TrailerTakes', 'th...",NaN,"{'total': 16, 'highlighted': [{'name': 'New Di...","{'coming_soon': False, 'date': 'Aug 7, 2019'}","{'url': 'https://twitter.com/psikodelya', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [1, 5], 'notes': 'This Game may contai..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6642,game,像素修真 - Pixel Xiuzhen,907240,0.0,False,NaN,NaN,"<strong><h2 class=""bb_tag""><u>In v1.10, all th...","<strong><h2 class=""bb_tag""><u>In v1.10, all th...",Pixel Xiuzhen is a unique Roguelike single-pla...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 4, 'path_thumbnail': 'https://cdn.akam..

In [135]:
#This is to pull data from the steam spy website.
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data


# set files and columns
download_path = './data/download'
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    end=len(app_list),
    batchsize=50,
    pause=0.2
)

Starting at index 0:

Exported lines 0-4 to steamspy_data.csv. Batch 0 time: 0:00:05 (avg: 0:00:05, remaining: 14:50:31)
Exported lines 5-9 to steamspy_data.csv. Batch 1 time: 0:00:05 (avg: 0:00:05, remaining: 14:50:04)
Exported lines 10-14 to steamspy_data.csv. Batch 2 time: 0:00:05 (avg: 0:00:05, remaining: 14:50:25)
Exported lines 15-19 to steamspy_data.csv. Batch 3 time: 0:00:05 (avg: 0:00:05, remaining: 14:49:28)
Exported lines 20-24 to steamspy_data.csv. Batch 4 time: 0:00:05 (avg: 0:00:05, remaining: 14:48:59)
Exported lines 25-29 to steamspy_data.csv. Batch 5 time: 0:00:05 (avg: 0:00:05, remaining: 14:48:18)
Exported lines 30-34 to steamspy_data.csv. Batch 6 time: 0:00:05 (avg: 0:00:05, remaining: 14:45:48)
Exported lines 35-39 to steamspy_data.csv. Batch 7 time: 0:00:05 (avg: 0:00:05, remaining: 14:44:49)
Exported lines 40-44 to steamspy_data.csv. Batch 8 time: 0:00:05 (avg: 0:00:05, remaining: 14:44:06)
Exported lines 45-49 to steamspy_data.csv. Batch 9 time: 0:00:05 (avg: 0:

Exported lines 785-789 to steamspy_data.csv. Batch 157 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:46)
Exported lines 790-794 to steamspy_data.csv. Batch 158 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:42)
Exported lines 795-799 to steamspy_data.csv. Batch 159 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:36)
Exported lines 800-804 to steamspy_data.csv. Batch 160 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:31)
Exported lines 805-809 to steamspy_data.csv. Batch 161 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:29)
Exported lines 810-814 to steamspy_data.csv. Batch 162 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:24)
Exported lines 815-819 to steamspy_data.csv. Batch 163 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:17)
Exported lines 820-824 to steamspy_data.csv. Batch 164 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:09)
Exported lines 825-829 to steamspy_data.csv. Batch 165 time: 0:00:05 (avg: 0:00:05, remaining: 14:30:06)
Exported lines 830-834 to steamspy_data.csv. Batch 166 

Exported lines 1555-1559 to steamspy_data.csv. Batch 311 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:34)
Exported lines 1560-1564 to steamspy_data.csv. Batch 312 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:29)
Exported lines 1565-1569 to steamspy_data.csv. Batch 313 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:25)
Exported lines 1570-1574 to steamspy_data.csv. Batch 314 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:22)
Exported lines 1575-1579 to steamspy_data.csv. Batch 315 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:18)
Exported lines 1580-1584 to steamspy_data.csv. Batch 316 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:14)
Exported lines 1585-1589 to steamspy_data.csv. Batch 317 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:10)
Exported lines 1590-1594 to steamspy_data.csv. Batch 318 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:05)
Exported lines 1595-1599 to steamspy_data.csv. Batch 319 time: 0:00:05 (avg: 0:00:05, remaining: 14:19:00)
Exported lines 1600-1604 to steamspy_

Exported lines 1940-1944 to steamspy_data.csv. Batch 388 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:45)
Exported lines 1945-1949 to steamspy_data.csv. Batch 389 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:42)
Exported lines 1950-1954 to steamspy_data.csv. Batch 390 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:38)
Exported lines 1955-1959 to steamspy_data.csv. Batch 391 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:55)
Exported lines 1960-1964 to steamspy_data.csv. Batch 392 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:53)
Exported lines 1965-1969 to steamspy_data.csv. Batch 393 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:51)
Exported lines 1970-1974 to steamspy_data.csv. Batch 394 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:48)
Exported lines 1975-1979 to steamspy_data.csv. Batch 395 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:44)
Exported lines 1980-1984 to steamspy_data.csv. Batch 396 time: 0:00:05 (avg: 0:00:05, remaining: 14:16:40)
Exported lines 1985-1989 to steamspy_

Exported lines 2325-2329 to steamspy_data.csv. Batch 465 time: 0:00:05 (avg: 0:00:05, remaining: 14:13:06)
Exported lines 2330-2334 to steamspy_data.csv. Batch 466 time: 0:00:05 (avg: 0:00:05, remaining: 14:13:02)
Exported lines 2335-2339 to steamspy_data.csv. Batch 467 time: 0:00:05 (avg: 0:00:05, remaining: 14:12:58)
Exported lines 2340-2344 to steamspy_data.csv. Batch 468 time: 0:00:05 (avg: 0:00:05, remaining: 14:12:55)
Exported lines 2345-2349 to steamspy_data.csv. Batch 469 time: 0:00:05 (avg: 0:00:05, remaining: 14:12:50)
Exported lines 2350-2354 to steamspy_data.csv. Batch 470 time: 0:00:05 (avg: 0:00:05, remaining: 14:12:46)
Exported lines 2355-2359 to steamspy_data.csv. Batch 471 time: 0:00:05 (avg: 0:00:05, remaining: 14:12:44)
Exported lines 2360-2364 to steamspy_data.csv. Batch 472 time: 0:00:05 (avg: 0:00:05, remaining: 14:12:40)
Exported lines 2365-2369 to steamspy_data.csv. Batch 473 time: 0:00:05 (avg: 0:00:05, remaining: 14:12:36)
Exported lines 2370-2374 to steamspy_

...

Exported lines 54960-54964 to steamspy_data.csv. Batch 10992 time: 0:00:05 (avg: 0:00:05, remaining: 0:26:17)
Exported lines 54965-54969 to steamspy_data.csv. Batch 10993 time: 0:00:05 (avg: 0:00:05, remaining: 0:26:12)
Exported lines 54970-54974 to steamspy_data.csv. Batch 10994 time: 0:00:05 (avg: 0:00:05, remaining: 0:26:08)
Exported lines 54975-54979 to steamspy_data.csv. Batch 10995 time: 0:00:05 (avg: 0:00:05, remaining: 0:26:03)
Exported lines 54980-54984 to steamspy_data.csv. Batch 10996 time: 0:00:05 (avg: 0:00:05, remaining: 0:25:58)
Exported lines 54985-54989 to steamspy_data.csv. Batch 10997 time: 0:00:05 (avg: 0:00:05, remaining: 0:25:53)
Exported lines 54990-54994 to steamspy_data.csv. Batch 10998 time: 0:00:05 (avg: 0:00:05, remaining: 0:25:49)
Exported lines 54995-54999 to steamspy_data.csv. Batch 10999 time: 0:00:05 (avg: 0:00:05, remaining: 0:25:44)
Exported lines 55000-55004 to steamspy_data.csv. Batch 11000 time: 0:00:05 (avg: 0:00:05, remaining: 0:25:39)
Exported l

Exported lines 55335-55339 to steamspy_data.csv. Batch 11067 time: 0:00:05 (avg: 0:00:05, remaining: 0:20:21)
Exported lines 55340-55344 to steamspy_data.csv. Batch 11068 time: 0:00:05 (avg: 0:00:05, remaining: 0:20:16)
Exported lines 55345-55349 to steamspy_data.csv. Batch 11069 time: 0:00:05 (avg: 0:00:05, remaining: 0:20:11)
Exported lines 55350-55354 to steamspy_data.csv. Batch 11070 time: 0:00:05 (avg: 0:00:05, remaining: 0:20:06)
Exported lines 55355-55359 to steamspy_data.csv. Batch 11071 time: 0:00:05 (avg: 0:00:05, remaining: 0:20:02)
Exported lines 55360-55364 to steamspy_data.csv. Batch 11072 time: 0:00:05 (avg: 0:00:05, remaining: 0:19:57)
Exported lines 55365-55369 to steamspy_data.csv. Batch 11073 time: 0:00:05 (avg: 0:00:05, remaining: 0:19:52)
Exported lines 55370-55374 to steamspy_data.csv. Batch 11074 time: 0:00:05 (avg: 0:00:05, remaining: 0:19:47)
Exported lines 55375-55379 to steamspy_data.csv. Batch 11075 time: 0:00:05 (avg: 0:00:05, remaining: 0:19:43)
Exported l

Exported lines 55710-55714 to steamspy_data.csv. Batch 11142 time: 0:00:05 (avg: 0:00:05, remaining: 0:14:24)
Exported lines 55715-55719 to steamspy_data.csv. Batch 11143 time: 0:00:05 (avg: 0:00:05, remaining: 0:14:20)
Exported lines 55720-55724 to steamspy_data.csv. Batch 11144 time: 0:00:05 (avg: 0:00:05, remaining: 0:14:15)
Exported lines 55725-55729 to steamspy_data.csv. Batch 11145 time: 0:00:05 (avg: 0:00:05, remaining: 0:14:10)
Exported lines 55730-55734 to steamspy_data.csv. Batch 11146 time: 0:00:05 (avg: 0:00:05, remaining: 0:14:05)
Exported lines 55735-55739 to steamspy_data.csv. Batch 11147 time: 0:00:05 (avg: 0:00:05, remaining: 0:14:01)
Exported lines 55740-55744 to steamspy_data.csv. Batch 11148 time: 0:00:05 (avg: 0:00:05, remaining: 0:13:56)
Exported lines 55745-55749 to steamspy_data.csv. Batch 11149 time: 0:00:05 (avg: 0:00:05, remaining: 0:13:51)
Exported lines 55750-55754 to steamspy_data.csv. Batch 11150 time: 0:00:05 (avg: 0:00:05, remaining: 0:13:46)
Exported l

Exported lines 56085-56089 to steamspy_data.csv. Batch 11217 time: 0:00:05 (avg: 0:00:05, remaining: 0:08:28)
Exported lines 56090-56094 to steamspy_data.csv. Batch 11218 time: 0:00:05 (avg: 0:00:05, remaining: 0:08:23)
Exported lines 56095-56099 to steamspy_data.csv. Batch 11219 time: 0:00:05 (avg: 0:00:05, remaining: 0:08:19)
Exported lines 56100-56104 to steamspy_data.csv. Batch 11220 time: 0:00:05 (avg: 0:00:05, remaining: 0:08:14)
Exported lines 56105-56109 to steamspy_data.csv. Batch 11221 time: 0:00:05 (avg: 0:00:05, remaining: 0:08:09)
Exported lines 56110-56114 to steamspy_data.csv. Batch 11222 time: 0:00:05 (avg: 0:00:05, remaining: 0:08:04)
Exported lines 56115-56119 to steamspy_data.csv. Batch 11223 time: 0:00:05 (avg: 0:00:05, remaining: 0:08:00)
Exported lines 56120-56124 to steamspy_data.csv. Batch 11224 time: 0:00:05 (avg: 0:00:05, remaining: 0:07:55)
Exported lines 56125-56129 to steamspy_data.csv. Batch 11225 time: 0:00:05 (avg: 0:00:05, remaining: 0:07:50)
Exported l

Exported lines 56460-56464 to steamspy_data.csv. Batch 11292 time: 0:00:05 (avg: 0:00:05, remaining: 0:02:32)
Exported lines 56465-56469 to steamspy_data.csv. Batch 11293 time: 0:00:05 (avg: 0:00:05, remaining: 0:02:27)
Exported lines 56470-56474 to steamspy_data.csv. Batch 11294 time: 0:00:05 (avg: 0:00:05, remaining: 0:02:22)
Exported lines 56475-56479 to steamspy_data.csv. Batch 11295 time: 0:00:05 (avg: 0:00:05, remaining: 0:02:18)
Exported lines 56480-56484 to steamspy_data.csv. Batch 11296 time: 0:00:05 (avg: 0:00:05, remaining: 0:02:13)
Exported lines 56485-56489 to steamspy_data.csv. Batch 11297 time: 0:00:05 (avg: 0:00:05, remaining: 0:02:08)
Exported lines 56490-56494 to steamspy_data.csv. Batch 11298 time: 0:00:05 (avg: 0:00:05, remaining: 0:02:03)
Exported lines 56495-56499 to steamspy_data.csv. Batch 11299 time: 0:00:05 (avg: 0:00:05, remaining: 0:01:59)
Exported lines 56500-56504 to steamspy_data.csv. Batch 11300 time: 0:00:05 (avg: 0:00:05, remaining: 0:01:54)
Exported l

In [172]:
#Checking the data from the steamspy_data; no issues here
steamspy = pd.read_csv('./data/download/steamspy_data.csv', error_bad_lines=False, skip_blank_lines=False)
steamspy

C:\Users\charl\AppData\Local\Temp\ipykernel_22212\3679551604.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  steamspy = pd.read_csv('./data/download/steamspy_data.csv', error_bad_lines=False, skip_blank_lines=False)


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,200165,5162,0,"10,000,000 .. 20,000,000",11836,2244,203,799,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,11306,"{'Action': 5425, 'FPS': 4830, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,5788,928,0,"5,000,000 .. 10,000,000",99,2,17,2,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,80,"{'Action': 753, 'FPS': 311, 'Multiplayer': 263..."
2,30,Day of Defeat,Valve,Valve,NaN,5222,564,0,"5,000,000 .. 10,000,000",111,0,17,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain",Action,148,"{'FPS': 791, 'World War II': 255, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1953,434,0,"5,000,000 .. 10,000,000",115,0,8,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,5,"{'Action': 631, 'FPS': 142, 'Classic': 109, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,14743,746,0,"5,000,000 .. 10,000,000",265,0,210,0,499.0,499.0,0.0,"English, French, German, Korean",Action,72,"{'FPS': 893, 'Action': 334, 'Classic': 262, 'S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56618,2135580,Again and Again,Perspektik,Perspektik,NaN,15,0,0,"0 .. 20,000",0,0,0,0,499.0,499.0,0.0,English,Indie,2,"{'Thriller': 44, 'Horror': 38, 'Psychological ..."
56619,2135940,Light Eater,Austin Chase Williams,Austin Chase Williams,NaN,1,0,0,"0 .. 20,000",0,0,0,0,499.0,499.0,0.0,English,Indie,0,"{'Singleplayer': 44, 'Horror': 37, 'Puzzle': 3..."
56620,2137350,Input Chaos,anthony undreiner,anthony undreiner,NaN,2,0,0,"0 .. 20,000",0,0,0,0,999.0,999.0,0.0,"English, French, Spanish - Spain, Japanese","Action, Indie",1,"{'Action': 96, 'Spectacle fighter': 88, 'Robot..."
56621,2137570,Cairn Stone Balancing,Ammonite Design Studios Ltd,Ammonite Design Studios Ltd,NaN,2,3,0,"0 .. 20,000",0,0,0,0,799.0,799.0,0.0,English,"Casual, Indie, Simulation, Strategy",0,"{'Casual': 73, 'Simulation': 67, 'Strategy': 6..."


In [186]:
#This is to merge all the steamappdatas into one dataframe, it produced a lot of unnamed columns but that's fine we'll delete it in the cleaning sheet
steamdf = steamappdata.append(steamappdata2).append(steamappdata3)
steamdf

C:\Users\charl\AppData\Local\Temp\ipykernel_22212\1920180492.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  steamdf = steamappdata.append(steamappdata2).append(steamappdata3)


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,Unnamed: 261,Unnamed: 262,Unnamed: 263,Unnamed: 264,Unnamed: 265,Unnamed: 266,Unnamed: 267,Unnamed: 268,Unnamed: 269,Unnamed: 270
0,game,Counter-Strike,10,0,FALSE,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,Team Fortress Classic,20,0,FALSE,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,game,Day of Defeat,30,0,FALSE,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,game,Deathmatch Classic,40,0,FALSE,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,game,Half-Life: Opposing Force,50,0,FALSE,NaN,NaN,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6642,game,像素修真 - Pixel Xiuzhen,907240,0.0,False,NaN,NaN,"<strong><h2 class=""bb_tag""><u>In v1.10, all th...","<strong><h2 class=""bb_tag""><u>In v1.10, all th...",Pixel Xiuzhen is a unique Roguelike single-pla...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6643,game,GUNGRAVE VR U.N,907250,13.0,False,NaN,NaN,A standalone GUNGRAVE VR adventure featuring n...,A standalone GUNGRAVE VR adventure featuring n...,Become the legendary undead gunman Beyond the ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6644,game,eSports Legend,907270,0.0,False,NaN,NaN,Esports Legend is a Esports club simulator. Yo...,Esports Legend is a Esports club simulator. Yo...,Esports Legend is a Esports club simulator. Yo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6645,game,World of Myths,907310,0.0,True,NaN,NaN,<h1>Egyptian Mythology Expansion</h1><p><img s...,"Take control of the battle, crush the opponent...","Take control of the battle, crush the opponent...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
#Saving the final query sheet into the download part
#steamdf.to_csv('./data/download/steamdf.csv', index=False)